BDLE 2022

date du document  :  30/09/2022

# TP1 2022 Préparation de données ETU


Préparation de données


## Préparation du TP

Vérifier que des ressources de calcul sont allouées à votre notebook est connecté (cf RAM  de disque indiqués en haut à droite) . Sinon cliquer sur le bouton connecter pour obtenir des ressources.




Pour accéder directement aux fichiers stockées sur votre google drive. Renseigner le code d'authentification lorsqu'il est demandé

Ajuster le nom de votre dossier : MyDrive/ens/bdle/DM1

In [1]:
# import os
# from google.colab import drive
# drive.mount("/content/drive")

# drive_dir = "/content/drive/MyDrive/ens/bdle/TP1"
# os.makedirs(drive_dir, exist_ok=True)
# os.listdir(drive_dir)

Installer pyspark et findspark :


In [2]:
!pip install -q pyspark
!pip install -q findspark

     |████████████████████████████████| 281.3 MB 49 kB/s 
     |████████████████████████████████| 199 kB 23.6 MB/s 


Démarrer la session spark

In [3]:
import os

# !find /usr/local/lib -name "pyspark"
os.environ["SPARK_HOME"] = "/usr/local/lib/python3.7/dist-packages/pyspark"
os.environ["JAVA_HOME"] = "/usr"

In [4]:
# Principaux import
import findspark
from pyspark.sql import SparkSession 
from pyspark import SparkConf  

# pour les dataframe et udf
from pyspark.sql import *  
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import *

# pour le chronomètre
import time

# initialise les variables d'environnement pour spark
findspark.init()

# Démarrage session spark 
# --------------------------
def demarrer_spark():
  local = "local[*]"
  appName = "TP"
  configLocale = SparkConf().setAppName(appName).setMaster(local).\
  set("spark.executor.memory", "6G").\
  set("spark.driver.memory","6G").\
  set("spark.sql.catalogImplementation","in-memory")
  
  spark = SparkSession.builder.config(conf = configLocale).getOrCreate()
  sc = spark.sparkContext
  sc.setLogLevel("ERROR")
  
  spark.conf.set("spark.sql.autoBroadcastJoinThreshold","-1")

  # On ajuste l'environnement d'exécution des requêtes à la taille du cluster (4 coeurs)
  spark.conf.set("spark.sql.shuffle.partitions","4")    
  print("session démarrée, son id est ", sc.applicationId)
  return spark
spark = demarrer_spark()

session démarrée, son id est  local-1665933541805


In [5]:
# on utilise 8 partitions au lieu de 200 par défaut
spark.conf.set("spark.sql.shuffle.partitions", "8")
print("Nombre de partitions utilisées : ", spark.conf.get("spark.sql.shuffle.partitions"))

Nombre de partitions utilisées :  8


In [6]:
# Optionnel :
# pour l'accès à spark UI : voir https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/
# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !unzip ngrok-stable-linux-amd64.zip
# get_ipython().system_raw('./ngrok http 4050 &')
# !curl -s http://localhost:4040/api/tunnels

Redéfinir la fonction **display** pour afficher le resutltat des requêtes dans un tableau

In [7]:
import pandas as pd
from google.colab import data_table

# alternatives to Databricks display function.

def display(df, n=100):
  return data_table.DataTable(df.limit(n).toPandas(), include_index=False, num_rows_per_page=10)

def display2(df, n=20):
  pd.set_option('max_columns', None)
  pd.set_option('max_colwidth', None)
  return df.limit(n).toPandas()


Définir le tag **%%sql** pour pouvoir écrire plus simplement des requêtes en SQL dans une cellule

In [8]:
from IPython.core.magic import (register_line_magic, register_cell_magic, register_line_cell_magic)

def removeComments(query):
  result = ""
  for line in query.split('\n'):
    if not(line.strip().startswith("--")):
      result += line + "\n"
  return result

@register_line_cell_magic
def sql(line, cell=None):
    "To run a sql query. Use:  %%sql"
    val = cell if cell is not None else line
    tabRequetes = removeComments(val).split(";")
    derniere = None
    est_requete = False
    for r in tabRequetes:
        r = r.strip()
        if len(r) > 2:
          derniere = spark.sql(r)
          est_requete = r.lower().startswith('select')
    if(est_requete):
      return display(derniere)
    else:
      return print('ok')

In [9]:
# facultatif (à ne pas utiliser)
# %load_ext google.colab.data_table
# %unload_ext google.colab.data_table

## Accès aux données

### URL pour l'accès aux datasets

In [10]:
# URL du dossier PUBLIC_DATASET contenant des fichiers de données pour les TP
# ---------------------------------------------------------------------------
# en cas de problème avec le téléchargement des datasets, aller directement sur l'URL ci-dessous
PUBLIC_DATASET_URL = "https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4" 
PUBLIC_DATASET=PUBLIC_DATASET_URL + "/download?path="

print("URL du dossier contenant les datasets ", PUBLIC_DATASET_URL)

URL du dossier contenant les datasets  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4


### Données de mobilité

Données issues du dataset YFCC

In [11]:
local_dir = "/local/data"
os.makedirs(local_dir, exist_ok=True)
os.listdir(local_dir)

[]

In [12]:
from urllib import request

# download dataset if not already donwloaded
def download_file(web_dir, local_dir, file):
  local_file = local_dir + "/" + file
  web_file = web_dir + "/" + file
  if(os.path.isfile(local_file)):
    print(file, "is already stored")
  else:
    print("downloading from URL: ", web_file , "save in : " + local_file)
    request.urlretrieve(web_file , local_file)

# user visits
web_dir = PUBLIC_DATASET + "YFCC_POI_dataset_K_H_LIM/dataset_IJCAI_2015/data-ijcai15/userVisits-ijcai15"
download_file(web_dir, local_dir, "userVisits-Toro.csv")

# poi 
web_dir = PUBLIC_DATASET + "YFCC_POI_dataset_K_H_LIM/dataset_IJCAI_2015/data-ijcai15/poiList-ijcai15"
download_file(web_dir, local_dir, "POI-Toro.csv")



os.listdir(local_dir)

downloading from URL:  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=YFCC_POI_dataset_K_H_LIM/dataset_IJCAI_2015/data-ijcai15/userVisits-ijcai15/userVisits-Toro.csv save in : /local/data/userVisits-Toro.csv
downloading from URL:  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=YFCC_POI_dataset_K_H_LIM/dataset_IJCAI_2015/data-ijcai15/poiList-ijcai15/POI-Toro.csv save in : /local/data/POI-Toro.csv


['userVisits-Toro.csv', 'POI-Toro.csv']

### Les visites

Lire les 2 premières lignes du  fichier csv en python. 
Est ce que le fichier a une ligne d'entête ?
Quel caractère délimite deux valeurs consécutives dans une ligne de données ?

In [13]:
f = open(local_dir + "/" + "userVisits-Toro.csv", "r")
print(f.readline()); print(f.readline())

"photoID";"userID";"dateTaken";"poiID";"poiTheme";"poiFreq";"seqID"

7941504100;"10007579@N00";1346844688;30;"Structure";1538;1



Lire le fichier des visites *sans* préciser le type des attributs.
Par défaut, tous les attributs sont considérés comme étant de type string.

In [14]:
user_visits = spark.read.option("header", "True").option("delimiter", ";").format("csv").load(local_dir + "/" + "userVisits-Toro.csv")
user_visits.show(3)
user_visits.printSchema()

+----------+------------+----------+-----+---------+-------+-----+
|   photoID|      userID| dateTaken|poiID| poiTheme|poiFreq|seqID|
+----------+------------+----------+-----+---------+-------+-----+
|7941504100|10007579@N00|1346844688|   30|Structure|   1538|    1|
|4886005532|10012675@N05|1142731848|    6| Cultural|    986|    2|
|4886006468|10012675@N05|1142732248|    6| Cultural|    986|    2|
+----------+------------+----------+-----+---------+-------+-----+
only showing top 3 rows

root
 |-- photoID: string (nullable = true)
 |-- userID: string (nullable = true)
 |-- dateTaken: string (nullable = true)
 |-- poiID: string (nullable = true)
 |-- poiTheme: string (nullable = true)
 |-- poiFreq: string (nullable = true)
 |-- seqID: string (nullable = true)



Lire le fichier en précisant le schéma : nom et type des attributs

In [15]:
schema = "photoID long, userID String, date Long, poiID int, poiTheme String, poiFreq int, seqID int"

user_visits = spark.read.option("header", "True").option("delimiter", ";").csv(local_dir + "/" + "userVisits-Toro.csv", schema = schema)
user_visits.persist()
user_visits.createOrReplaceTempView("user_visits")
user_visits.show(10)
user_visits.printSchema()

+----------+------------+----------+-----+---------+-------+-----+
|   photoID|      userID|      date|poiID| poiTheme|poiFreq|seqID|
+----------+------------+----------+-----+---------+-------+-----+
|7941504100|10007579@N00|1346844688|   30|Structure|   1538|    1|
|4886005532|10012675@N05|1142731848|    6| Cultural|    986|    2|
|4886006468|10012675@N05|1142732248|    6| Cultural|    986|    2|
|4885404441|10012675@N05|1142732373|    6| Cultural|    986|    2|
|4886008334|10012675@N05|1142732445|    6| Cultural|    986|    2|
|4886009150|10012675@N05|1142916492|    6| Cultural|    986|    3|
|7054481539|10012675@N05|1319327174|   13| Cultural|    964|    4|
|6908387594|10012675@N05|1319328255|   13| Cultural|    964|    4|
|6908381912|10012675@N05|1319331463|   13| Cultural|    964|    4|
|6908398496|10012675@N05|1319331886|   13| Cultural|    964|    4|
+----------+------------+----------+-----+---------+-------+-----+
only showing top 10 rows

root
 |-- photoID: long (nullable = 

Vérifier qu'il n'y a pas de date nulle

In [16]:
%%sql
select count(1) as nb_dates_nulles
from user_visits
where date is null

-- autre solution avec "case"
--select count(case when isnull(date) then date end) as nb_date_nulles
--from user_visits

,nb_dates_nulles
0,0



Vérifier qu'il n'y a aucune séquence associée à plusieurs utilisateurs

In [17]:
%%sql
select seqID
from user_visits
group by seqID
having count(distinct userID) > 1

,seqID



#### POI_sequence

La table **POI_sequence**(seqID, poiID). Les POI visités durant une séquence.
Pour simplifier on compte chaque POI une seule fois par séquence et on ne considère pas l'ordre de visite des POI.

Rmq: il n'est pas nécessaire de préciser le userID pour identifier une séquence.

In [18]:
%%sql
create or replace temp view POI_sequence as 
select distinct seqID, poiID
from user_visits;

select * from POI_sequence

,seqID,poiID
0,9,24
1,16,24
2,26,22
3,29,25
4,54,25
...,...,...
95,624,23
96,634,22
97,638,4
98,641,30


### Les lieux visités : POI
Ils sont appelés *Point Of Interest*

In [19]:
poi_schema = "poiID long, poiName String, latitude double, longitude double, theme String"

poi = spark.read.option("header", "True").option("delimiter", ";").csv(local_dir + "/" + "POI-Toro.csv", schema = poi_schema)
poi.show(3)
poi.printSchema()
poi.createOrReplaceTempView("POI")

+-----+------------------+--------+---------+-----+
|poiID|           poiName|latitude|longitude|theme|
+-----+------------------+--------+---------+-----+
|    1| Air_Canada_Centre|43.64333|-79.37917|Sport|
|    2|         BMO_Field|43.63278|-79.41861|Sport|
|    3|Maple_Leaf_Gardens|43.66222|-79.38028|Sport|
+-----+------------------+--------+---------+-----+
only showing top 3 rows

root
 |-- poiID: long (nullable = true)
 |-- poiName: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- theme: string (nullable = true)



In [20]:
%%sql
cache table POI;

SELECT * 
FROM POI

,poiID,poiName,latitude,longitude,theme
0,1,Air_Canada_Centre,43.643330,-79.379170,Sport
1,2,BMO_Field,43.632780,-79.418610,Sport
2,3,Maple_Leaf_Gardens,43.662220,-79.380280,Sport
3,4,Rogers_Centre,43.641390,-79.389170,Sport
4,5,Woodbine_Racetrack,43.712525,-79.602042,Sport
5,6,Art_Gallery_of_Ontario,43.653890,-79.392780,Cultural
6,7,Hockey_Hall_of_Fame,43.646976,-79.377253,Cultural
7,8,Ripley%27s_Aquarium_of_Canada,43.642481,-79.386050,Cultural
8,9,Ontario_Science_Centre,43.716670,-79.338330,Cultural
9,10,Riverdale_Farm,43.667111,-79.361294,Cultural


la liste des thèmes

In [21]:
%%sql
create or replace temp view themes as
select distinct theme
from POI
order by theme;

cache table themes;

select *
from themes

,theme
0,Amusement
1,Beach
2,Cultural
3,Shopping
4,Sport
5,Structure


## Exercice 1

#### 1) Les 10 POI les plus photographiés
indications: on peut compléter une requête SQL avec *LIMIT 10* pour borner la taille résultat (partiel) souhaité.

In [22]:
%%sql
select distinct p.poiName, u.poiFreq 
from POI p, user_visits u
where p.poiID = u.poiID
order by u.poiFreq desc 
limit 10;

,poiName,poiFreq
0,Royal_Ontario_Museum,4142
1,Nathan_Phillips_Square,3619
2,Yonge-Dundas_Square,3594
3,CN_Tower,3553
4,Air_Canada_Centre,3506
5,Rogers_Centre,3056
6,Hockey_Hall_of_Fame,2064
7,Toronto_Eaton_Centre,1874
8,Ripley%27s_Aquarium_of_Canada,1736
9,Kensington_Market,1701


#### 2) Les visites avec date détaillée
Définir la table Visite_date(userID, seqID, poiID, date, annee, mois, jour, heure) contenant les visites avec la date détaillée composée des attributs : année, mois, jour, heure.
Indication : la date est initialement au format "unix". Voir la fonction de conversion de date from_unixtime et la fonction extract.

In [23]:
%%sql
create or replace temp view Visite_Date as

select distinct u.userID, u.seqID, p.poiName, from_unixtime(u.date) as POI_date, from_unixtime(u.date, "yyyy") as annee,
from_unixtime(u.date, "MM") as mois, from_unixtime(u.date, "dd") as jour, from_unixtime(u.date, "HH") as heure,
from_unixtime(u.date, "mm") as minute, from_unixtime(u.date, "ss") as seconde
from POI p, user_visits u
where p.poiID = u.poiID
order by seqID asc;

select * from Visite_Date

,userID,seqID,poiName,POI_date,annee,mois,jour,heure,minute,seconde
0,10007579@N00,1,Union_Station_(Toronto),2012-09-05 11:31:28,2012,09,05,11,31,28
1,10012675@N05,2,Art_Gallery_of_Ontario,2006-03-19 01:37:28,2006,03,19,01,37,28
2,10012675@N05,2,Art_Gallery_of_Ontario,2006-03-19 01:30:48,2006,03,19,01,30,48
3,10012675@N05,2,Art_Gallery_of_Ontario,2006-03-19 01:39:33,2006,03,19,01,39,33
4,10012675@N05,2,Art_Gallery_of_Ontario,2006-03-19 01:40:45,2006,03,19,01,40,45
...,...,...,...,...,...,...,...,...,...,...
95,10014440@N06,10,Kensington_Market,2008-07-09 08:30:44,2008,07,09,08,30,44
96,10014440@N06,10,Kensington_Market,2008-07-09 08:50:41,2008,07,09,08,50,41
97,10014440@N06,10,Kensington_Market,2008-07-09 10:01:51,2008,07,09,10,01,51
98,10014440@N06,10,Kensington_Market,2008-07-09 10:21:58,2008,07,09,10,21,58


#### 3a) Le nombre de POI par utilisateur
Le résultat (userID, nbPOI) est trié par nombre décroissant de POI.

In [24]:
%%sql
select u.userID, count( distinct u.poiID) as nbPOI
from user_visits u
group by u.userID
order by nbPOI desc

,userID,nbPOI
0,34211328@N00,24
1,43139087@N00,23
2,84987970@N00,22
3,30624156@N00,20
4,63677124@N07,19
...,...,...
95,16048448@N00,8
96,31349854@N00,8
97,56020405@N08,8
98,36101699447@N01,8


#### 3b) Le nombre de POI par séquence
Définir la table Seq3plus(seqID, nbPoi) correspondant aux séquences qui contiennent au moins 3 POI distincts.
Afficher le résultat trié par nombre de POI décroissant puis numéro de séquence croissant.


In [25]:
%%sql
create or replace temp view Seq3plus as 

select seqID, count( distinct poiID) as nbPOI
from user_visits
group by seqID
having nbPOI >=3;

select * from seq3plus
order by nbPOI desc, seqID;

,seqID,nbPOI
0,298,13
1,4961,10
2,4351,9
3,5964,9
4,510,8
...,...,...
95,2377,4
96,2466,4
97,2472,4
98,2896,4


#### 4) Trajectoire



Définir la table Trajectoire(userID, seqID, listPOI).

listPOI est la liste des POI visités, pendant la séquence seqID, dans l'ordre chronologique

Indications:
pour la sequence 687 la listPOI est [7,16,4,8,4,16]

In [26]:
from itertools import groupby


def trier_poi(listePOI):
  """
  Fonction pour trier par ordre chronologique les POI visités.
  
  @param listePOI: liste des POI avec dates [[poi1, date1], [poi2, date2]...]

  @return liste du POI trié [poi1, poi2...]
  """

  # Tri les POI par ordre chronologique
  result = sorted(listePOI, key=lambda x: x[1])
  
  # Map aux POI
  result = list(map(lambda x: x[0], result))

  # Elimine les duplications consecutives 
  result = [key for key, _group in groupby(result)]
  
  return result


# Enregistre le trier_poi udf qui retourne un Array de type Integer
spark.udf.register('trier_poi', trier_poi, ArrayType(IntegerType()))

# test local
print(trier_poi([[13,99],[10,5], [12,4], [14,2],[11,1]]))

[11, 14, 12, 10, 13]


In [27]:
%%sql

create or replace temp view Trajectoire as

select userID, seqID, trier_poi(arrays_zip(array_agg(poiID), array_agg(date))) as listePOI
from user_visits
group by seqID, userID;

cache table Trajectoire;

select * from Trajectoire
where size(listePOI) >6
order by seqID

,userID,seqID,listePOI
0,13644648@N03,298,"[22, 7, 23, 28, 1, 29, 30, 8, 29, 6, 28, 23, 3..."
1,14878709@N00,454,"[16, 21, 23, 16, 8, 16, 8, 16, 8, 16, 28, 22, ..."
2,16048448@N00,510,"[28, 22, 23, 29, 7, 30, 16, 21]"
3,16693950@N00,525,"[22, 8, 30, 7, 28, 22, 28, 23, 21, 30, 7, 8, 1..."
4,17373191@N00,549,"[22, 1, 8, 21, 22, 28, 22]"
5,20342758@N00,681,"[23, 28, 16, 24, 4, 8, 30, 8, 16, 24]"
6,20456447@N03,686,"[22, 30, 7, 1, 8, 16, 1, 21]"
7,20741443@N00,708,"[7, 23, 28, 23, 21, 30, 7]"
8,20741443@N00,809,"[7, 30, 21, 23, 21, 23, 28, 7]"
9,20741443@N00,824,"[21, 23, 21, 23, 21, 23, 21]"


verification 

In [40]:
 %%sql
 select *
 from user_Visits
 where seqID = 298
 order by date

,photoID,userID,date,poiID,poiTheme,poiFreq,seqID
0,9075435841,13644648@N03,1371514467,22,Beach,3619,298
1,9077664280,13644648@N03,1371514467,22,Beach,3619,298
2,9075436255,13644648@N03,1371514471,7,Cultural,2064,298
3,9077664846,13644648@N03,1371514472,7,Cultural,2064,298
4,9075747195,13644648@N03,1371516913,23,Shopping,1874,298
5,9077974672,13644648@N03,1371516914,28,Structure,936,298
6,9077974802,13644648@N03,1371516915,28,Structure,936,298
7,9077974776,13644648@N03,1371516915,28,Structure,936,298
8,9078023234,13644648@N03,1371517315,1,Sport,3506,298
9,9075796819,13644648@N03,1371517318,30,Structure,1538,298


#### 5) Transitions
5a) Définir la table Transitions(poi1, poi2, nbTrans) avec nbTrans étant le nombre de déplacements directs de poi1 à poi2.
Indication: deux POI apparaissant successivement (poi1 suivi de poi2) dans une séquence forment un déplacement direct entre poi1 et poi2.
Une séquence contenant un seul POI ne correspond à aucun déplacement.

In [29]:
from itertools import groupby


def transition(listePOI):
  """
    Calcule les transitions des POI quand donné un array en forme [poi1, poi2...]

    @param listePOI: array de IntegerType() [POI1, POI2...]

    @return array [[poi1,poi2],[poi2,poi3]...]
  """ 
  
  # Créer les transitions
  result = [[poi, listePOI[index+1]] for index, poi in enumerate(listePOI) if index + 1 < len(listePOI)]

  return result

spark.udf.register("transition", transition, ArrayType(ArrayType(IntegerType())))

# test local
print(transition([2,6,1,6]))

[[2, 6], [6, 1], [1, 6]]


In [30]:
%%sql

create or replace temp view Transition1 as

select seqID, transition[0] as p1, transition[1] as p2
from (
  select seqID, explode(transition(listePOI)) as transition
  from Trajectoire
  );

select seqID, p1, p2
from Transition1 

,seqID,p1,p2
0,111,30,1
1,157,27,23
2,157,23,11
3,157,11,27
4,166,7,22
...,...,...,...
95,1062,23,29
96,1066,16,8
97,1066,8,16
98,1066,16,30


In [31]:
%%sql
create or replace temp view Transition2 as 

select p1,p2,count(*) as nbtransition
from Transition1
group by p1,p2;

select * from Transition2
order by nbtransition desc;

,p1,p2,nbtransition
0,23,21,82
1,22,28,70
2,28,22,64
3,21,23,63
4,7,30,57
...,...,...,...
95,13,16,4
96,11,25,4
97,4,22,4
98,29,30,4


b) Definir la vue 
Transition_relative avec des valeurs de transition relatives au nombre total de transition partant d'un POI.

Indication: créer une vue donnant le nombre total de transitions par poi.

In [32]:
%%sql
create or replace temp view Total_transition as 

select p1, sum(nbtransition) as total
from Transition2
group by p1;


select * from Total_transition

,p1,total
0,13,20
1,14,16
2,23,211
3,16,166
4,6,61
5,17,12
6,9,1
7,10,4
8,11,68
9,7,153


In [33]:
%%sql
create or replace temp view Transition_relative as 

select tr.p1, tr.p2, tr.nbTransition/to.total
from Transition2 as tr
join Total_transition as to on tr.p1 = to.p1 ;



select * from Transition_relative
order by  p1

,p1,p2,(nbTransition / total)
0,1,7,0.111111
1,1,16,0.088889
2,1,30,0.311111
3,1,21,0.088889
4,1,8,0.155556
...,...,...,...
95,11,7,0.029412
96,13,4,0.050000
97,13,11,0.300000
98,13,28,0.050000


#### Distances entre POIs
table Distance(poi1, poi2, distance)

In [34]:
%%sql

create or replace temp view POI_coord as

select poiID, latitude, longitude
from poi;

select 
  poi1, 
  poi2, 
  (1.609344 * 6371 * 2  * asin(sqrt((power(sin(((lat2/180/(3.141592)) - (lat1/180/(3.141592)))/2),2) + cos((lat1/180/(3.141592))) * cos((lat2/180/(3.141592))) 
  * power(sin(((long2/180/(3.141592))-(long1/180/(3.141592)))/2),2))))) as distance 
from (
  select 
    p1.poiID as poi1, p1.latitude as lat1, p1.longitude as long1,
    p2.poiID as poi2, p2.latitude as lat2, p2.longitude as long2 
  from POI_coord as p1 cross join POI_coord as p2
  order by poi1, poi2
  );


,poi1,poi2,distance
0,1,1,0.000000
1,1,2,0.738194
2,1,3,0.343092
3,1,4,0.184166
4,1,5,4.219777
...,...,...,...
95,4,6,0.235852
96,4,7,0.238051
97,4,8,0.059770
98,4,9,1.645518


### 6) DuréeVisitePOI 
La durée moyenne de visite d'un POI.
Indication, pour une série d'événements consécutifs associés à un même POI dans une séquence, la durée de visite est la différence de date entre le 1er et le dernier événement.

In [35]:
%%sql

select poiID, avg(max - min) as duration
from (select userID, seqID, poiID, min(date) as min, max(date) as max
from user_visits
group by userID, poiID, seqID)
group by poiID;

,poiID,duration
0,13,2124.133333
1,12,3136.986486
2,14,965.973684
3,18,1037.400000
4,6,1379.530435
5,23,1468.289225
6,16,2971.145068
7,17,6392.653846
8,9,3673.694444
9,10,2013.134146


## Exercice 2

Charger les données de Geonames

In [36]:
import zipfile

#geonames
web_dir = PUBLIC_DATASET + "geonames_ALL"

download_file(web_dir, local_dir, "allCountries.zip")

#unzip
local_file = local_dir + "/" + "allCountries.txt"
if(os.path.isfile(local_file)):
    print(local_file, "is already stored")
else:
  with zipfile.ZipFile(local_dir + "/" + "allCountries.zip", 'r') as zip_ref:
    zip_ref.extractall(local_dir)

os.listdir(local_dir)

downloading from URL:  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=geonames_ALL/allCountries.zip save in : /local/data/allCountries.zip


['allCountries.txt', 'userVisits-Toro.csv', 'allCountries.zip', 'POI-Toro.csv']

In [37]:
geonames =  spark.read.option("header", "False").option("delimiter", "\t").format("csv").load(local_dir + "/" + "allCountries.txt")
geonames.createOrReplaceTempView("geonames")
geonames.persist()
geonames.show(3)




+-------+--------------------+--------------------+--------------------+--------+-------+---+---+---+-----+----+----+----+----+----+----+----+--------------+----------+
|    _c0|                 _c1|                 _c2|                 _c3|     _c4|    _c5|_c6|_c7|_c8|  _c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|          _c17|      _c18|
+-------+--------------------+--------------------+--------------------+--------+-------+---+---+---+-----+----+----+----+----+----+----+----+--------------+----------+
|2986043|  Pic de Font Blanca|  Pic de Font Blanca|Pic de Font Blanc...|42.64991|1.53335|  T| PK| AD| null|  00|null|null|null|   0|null|2860|Europe/Andorra|2014-11-05|
|2994701|            Roc Mélé|            Roc Mele|Roc Mele,Roc Mele...|42.58765|1.74028|  T| MT| AD|AD,FR|  00|null|null|null|   0|null|2803|Europe/Andorra|2014-11-05|
|3007683|Pic des Langounelles|Pic des Langounelles|Pic des Langounelles|42.61203|1.47364|  T| PK| AD|AD,FR|  00|null|null|null|   0|null|2685|Europe/Andorr

### 1) Geonames


#####1a) schéma de Geonames

Définir Geonames2 en précisant le schéma : nom et type des attributs.
Se limiter au 9 premiers attributs donnant des informations sur l'identifiant d'un POI, ses noms, sa position GPS, le code du pays..., ), d'après le document [readme.txt](https://nuage.lip6.fr/s/kaBX3sF3YEzMZHK) dans le dossier [geonames_ALL](https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4?path=%2Fgeonames_ALL)

In [38]:
%%sql
create or replace temp view Geonames2 as 
select cast(_c0 as long) as geonameID, 
  cast(_c1 as varchar(200)) as name, 
  cast(_c2 as varchar(200)) as asciiname, 
  (cast(_c2 as varchar(200)), cast(_c3 as varchar(200))) as alternate_name, 
  cast(_c4 as FLOAT) as latitude, 
  cast(_c5 as FLOAT) as longitude, 
  cast(_c6 as char(1)) as feature_class, 
  cast(_c7 as varchar(10)) as feature_code,
  cast(_c8 as char(2)) as country_code 

from Geonames;

select * from Geonames2

,geonameID,name,asciiname,alternate_name,latitude,longitude,feature_class,feature_code,country_code
0,2986043,Pic de Font Blanca,Pic de Font Blanca,"(Pic de Font Blanca, Pic de Font Blanca,Pic du...",42.649910,1.53335,T,PK,AD
1,2994701,Roc Mélé,Roc Mele,"(Roc Mele, Roc Mele,Roc Meler,Roc Mélé)",42.587650,1.74028,T,MT,AD
2,3007683,Pic des Langounelles,Pic des Langounelles,"(Pic des Langounelles, Pic des Langounelles)",42.612030,1.47364,T,PK,AD
3,3017832,Pic de les Abelletes,Pic de les Abelletes,"(Pic de les Abelletes, Pic de la Font-Negre,Pi...",42.525349,1.73343,T,PK,AD
4,3017833,Estany de les Abelletes,Estany de les Abelletes,"(Estany de les Abelletes, Estany de les Abelle...",42.529148,1.73362,H,LK,AD
...,...,...,...,...,...,...,...,...,...
95,3038902,Tosquers,Tosquers,"(Tosquers, None)",42.565868,1.48763,T,SLP,AD
96,3038903,Canal del Tosquer,Canal del Tosquer,"(Canal del Tosquer, None)",42.571140,1.52067,H,STM,AD
97,3038904,Bosc del Tosquer,Bosc del Tosquer,"(Bosc del Tosquer, None)",42.571892,1.52369,V,FRST,AD
98,3038905,Bosc del Tosquer,Bosc del Tosquer,"(Bosc del Tosquer, None)",42.537930,1.60070,V,FRST,AD


##### 1b) Extrait pour le Canada

Définir la vue Geonames_canada pour les POI situés au Canada.

In [41]:
%%sql
create or replace temp view Geonames_canada as
select geonameID, name, alternate_name, latitude, longitude, feature_class, feature_code from Geonames2
where country_code = 'CA';

cache table Geonames_canada;

select * from  Geonames_canada;

,geonameID,name,alternate_name,latitude,longitude,feature_class,feature_code
0,3424953,Virgin Rocks,"(Virgin Rocks, None)",46.428860,-50.819950,U,RFU
1,3425041,Eastern Shoals,"(Eastern Shoals, None)",46.447430,-50.485771,U,SHSU
2,3426061,Downing Basin,"(Downing Basin, None)",47.052158,-50.791630,U,TRGU
3,3831549,Kane Basin,"(Kane Basin, Bassin Kane,Kane Basin)",79.388603,-70.425407,H,STRT
4,3831631,Georges Shoal,"(Georges Shoal, Georges Bank,St. George's Bank...",41.654442,-67.732422,U,SHLU
...,...,...,...,...,...,...,...
95,5881729,Aalders Landing,"(Aalders Landing, None)",44.833450,-64.915489,L,AREA
96,5881730,Aalders Lang Brook,"(Aalders Lang Brook, None)",44.866852,-64.298752,H,STM
97,5881731,Aalders Lang Meadow,"(Aalders Lang Meadow, None)",44.850151,-64.315453,T,PLN
98,5881732,Aaltanhash Inlet,"(Aaltanhash Inlet, Aaltenash Bight)",53.128540,-128.517883,H,INLT


### 2) Association entre les POI et Geonames
Compléter des POI avec des attributs de geonames (par exemple name et [feature code](http://www.geonames.org/export/codes.html))

In [42]:
%%sql 

select p.poiID, p.poiName, g.alternate_name, g.feature_code
from POI p
join Geonames2 g on p.latitude - g.latitude <= 1 and p.longitude - g.longitude <= 1


,poiID,poiName,alternate_name,feature_code
0,1,Air_Canada_Centre,"(Pic de Font Blanca, Pic de Font Blanca,Pic du...",PK
1,1,Air_Canada_Centre,"(Port Vell, None)",PASS
2,1,Air_Canada_Centre,"(Serra de Tristaina, Serra de Tristaina)",RDGE
3,1,Air_Canada_Centre,"(Pic de Tristaina, Pic de Triatagne,Pic de Tri...",PK
4,1,Air_Canada_Centre,"(Port de Siguer, Port de Siguer)",PASS
...,...,...,...,...
95,1,Air_Canada_Centre,"(Brandstein, Brandstein)",MT
96,1,Air_Canada_Centre,"(Teufelsgraben, None)",STMI
97,1,Air_Canada_Centre,"(Zecker, None)",SLP
98,1,Air_Canada_Centre,"(Pfarrmauer, None)",CLF
